In [106]:
#Creditos a Alberto Herrera quien publico un tutorial el 29 ene. 2017 https://www.youtube.com/watch?v=VjN0cwVsYzE
import math
import numpy as np
from csv import reader
from scipy import optimize
from six.moves import input

In [107]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
    for row in dataset:
        row[column] = int(row[column]) 

In [108]:
class RedNeuronal(object):
    def __init__(self):
        #parametros
        #capa de entrada
        self.numeroNeuronasEntrada = 13
        #capa escondida
        self.numeroNeuronasEscondidas1 = 3
        self.numeroNeuronasEscondidas2 = 3
        #capa de salida
        self.numeroNeuronasSalida = 1

        #matriz con peso aleatorio
        self.W1 = np.random.rand(self.numeroNeuronasEntrada, self.numeroNeuronasEscondidas1)
        self.W2 = np.random.rand(self.numeroNeuronasEscondidas1, self.numeroNeuronasEscondidas2)
        self.W3 = np.random.rand(self.numeroNeuronasEscondidas2, self.numeroNeuronasSalida)

    def avanzar(self,x): #propagar
        #primero calculamos Z2
        self.Z2 = np.dot(x, self.W1)
        self.A2 = self.sigmoide(self.Z2)
        self.Z3 = np.dot(self.A2,self.W2)
        self.A3 = self.sigmoide(self.Z3)
        self.Z4 = np.dot(self.A3,self.W3)
        yprima = self.sigmoide(self.Z4)
        return yprima

    def sigmoide(self, z):
        return 1/(1 + np.exp(-z))

    def sigmoideprima(self, z):
        return np.exp(-z)/((1+np.exp(-z))**2)

    def funcionDeCosto(self, x, y):
        self.yprima = self.avanzar(x)
        J = 0.5*sum((y - self.yprima)**2)
        return J

    def funcionDeCostoPrima(self, x, y):#backpropagation
        self.yprima = self.avanzar(x)
        #error de los datos evaluados
        E = y - self.yprima
        delta3 = np.multiply(-(E) , self.sigmoideprima(self.Z4))
        djdw3 = np.dot(self.A3.T, delta3)
        delta2 = np.dot(delta3,self.W3.T) * self.sigmoideprima(self.Z3)
        djdw2 = np.dot(self.A2.T, delta2)
        delta1 = np.dot(delta2,self.W2.T) * self.sigmoideprima(self.Z2)
        djdw1 = np.dot(x.T, delta1)
        return djdw1, djdw2, djdw3

    def obtenerParametros(self):
        W1_vector = self.W1.ravel()
        W2_vector = self.W2.ravel()
        W3_vector = self.W3.ravel()
        parametros = np.concatenate((W1_vector,W2_vector,W3_vector))
        return parametros

    def setearParametros(self, parametros):#pasa de vector a matriz
        W1_start = 0
        W1_end = self.numeroNeuronasEntrada*self.numeroNeuronasEscondidas1
        W2_start = W1_end
        W2_end = W1_end + self.numeroNeuronasEscondidas1*self.numeroNeuronasEscondidas2
        W3_start = W2_end
        W3_end = W2_end + self.numeroNeuronasEscondidas2*self.numeroNeuronasSalida

        self.W1 = np.reshape(parametros[W1_start:W1_end],(self.numeroNeuronasEntrada,self.numeroNeuronasEscondidas1))
        self.W2 = np.reshape(parametros[W2_start:W2_end],(self.numeroNeuronasEscondidas1,self.numeroNeuronasEscondidas2))
        self.W3 = np.reshape(parametros[W3_start:W3_end],(self.numeroNeuronasEscondidas2,self.numeroNeuronasSalida))

    def calcularGradientes(self, x, y):
        djdw1, djdw2, djdw3 = self.funcionDeCostoPrima(x,y)
        djdw1_vector = djdw1.ravel()
        djdw2_vector = djdw2.ravel()
        djdw3_vector = djdw3.ravel()

        derivadas_vector = np.concatenate((djdw1_vector,djdw2_vector,djdw3_vector))
        return derivadas_vector

    def calcularGradientesNumericos(self, N, x, y):
        parametrosIniciales = N.obtenerParametros()
        resultados_gradiente_numerico = np.zeros(parametrosIniciales.shape)
        perturbaciones_epsilon = np. zeros(parametrosIniciales.shape)
        e = 1e-4

        for p in range(len(parametrosIniciales)):
            perturbaciones_epsilon[p] = e
            N.setearParametros(parametrosIniciales + perturbaciones_epsilon)
            perdida2 = N.funcionDeCosto(x,y)
            N.setearParametros(parametrosIniciales - perturbaciones_epsilon)
            perdida1 = N.funcionDeCosto(x,y)
            a = (perdida2 - perdida1)/(2*e)
            resultados_gradiente_numerico[p] = a
            perturbaciones_epsilon[p] = 0
            N.setearParametros(parametrosIniciales)
        return resultados_gradiente_numerico


In [110]:
class Entrenador(object):
    def __init__(self,N):
        self.N = N

    def costFunctionWrapper(self, params, X, y):
        self.N.setearParametros(params)
        cost = self.N.funcionDeCosto(X,y)
        grad = self.N.calcularGradientes(X,y)
        return cost, grad

    def callbackF(self,params):
        self.N.setearParametros(params)
        self.J.append(self.N.funcionDeCosto(self.X, self.y))
    
    def train(self, x, y):
        self.X = x
        self.y = y
        params = self.N.obtenerParametros()
        self.J = []
        _res = optimize.minimize(self.costFunctionWrapper, params, jac = True, method = 'BFGS', args= (x,y), options = {'disp' : True}, callback = self.callbackF)
        self.N.setearParametros(_res.x)
        self.optimizationResults = _res



In [113]:
filename = 'Train.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])-2):
    str_column_to_float(dataset, i+2)
for a in range(len(dataset[0])-2):
    str_column_to_int(dataset, a+2)
#prueba
redNeuronal = RedNeuronal()
redNeuronal.W1
redNeuronal.W2
redNeuronal.W3
#prueba propagacion
datox = []
datoy = []
numregistros=len(dataset)
print ("Numero de Registros de Entrenamiento: ", numregistros)
semanas = int(numregistros/168)
indi = 0
#for fila in range(semanas):
#    datamodel=dataset[indi] 
for fila in range(numregistros):
    datamodel=dataset[fila] 
    datox.append(datamodel[2:15])
    datoy.append(datamodel[15:16])
    indi=(fila+1)*168 #recorre solo el lunes a la hora 1
X = np.array((datox), dtype=float)
resultados = np.array((datoy),dtype=float)    
X = X/10000
resultados = resultados/10000
print ("La prediccion de la red neuronal es")
print (redNeuronal.avanzar(X))

#funcion de costo
print ("Los valores de la funcion de costo son")
print (redNeuronal.funcionDeCosto(X, resultados))

#backpropagation
a, b, c = redNeuronal.funcionDeCostoPrima(X,resultados)
#test gradiente
redNeuronal.obtenerParametros()
redNeuronal.calcularGradientes(X, resultados)
redNeuronal.calcularGradientesNumericos(redNeuronal,X,resultados)

#training
entrenador = Entrenador(redNeuronal)
entrenador.train(X, resultados)
print (redNeuronal.avanzar(X))


Numero de Registros de Entrenamiento:  6720
La prediccion de la red neuronal es
[[ 0.81944659]
 [ 0.8188909 ]
 [ 0.81867144]
 ..., 
 [ 0.82452683]
 [ 0.82254909]
 [ 0.82070097]]
Los valores de la funcion de costo son
[ 1456.39681601]
Optimization terminated successfully.
         Current function value: 0.048150
         Iterations: 1303
         Function evaluations: 1446
         Gradient evaluations: 1446
[[ 0.11382089]
 [ 0.11084112]
 [ 0.10879559]
 ..., 
 [ 0.15962541]
 [ 0.14201852]
 [ 0.12702628]]


In [117]:
#TEST1 entrada 1157,1157,1157,1178,1177,1178,1165,1168,1165,1118,1103,1118,1180 salida 1149 -> Lunes hora 1

T1 = np.array(([1157,1157,1157,1178,1177,1178,1165,1168,1165,1118,1103,1118,1180]), dtype=float)
T1 = T1/10000
print (redNeuronal.avanzar(T1)*10000)



[ 1154.42401501]


In [118]:
#TEST2 entrada 2006,2006,2006,2044,2020,2044,2003,2001,2003,1922,1915,1922,1959 salida 1977 -> Lunes hora 13 

T2 = np.array(([2006,2006,2006,2044,2020,2044,2003,2001,2003,1922,1915,1922,1959]), dtype=float)
T2 = T2/10000
print (redNeuronal.avanzar(T2)*10000)


[ 1972.93803227]
